In [187]:
%pip install pandas matplotlib seaborn scikit-learn -q
%pip install imbalanced-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [188]:

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('mcdonalds.csv')


In [189]:
df.head()

_id  active                      name code country  \
0  61d7fcbda283bebac83e89c9    True          SOBRADINHO - SBR  SBR      BR   
1  62ed34bb7877dc2d6381ea1c    True   Drive Saída Norte - DSN  DSN      BR   
2  62ed31417877dcd78481e3a7    True   Shopping Iguatemi - BSI  BSI      BR   
3  62ed30849b780128a34229f8    True  Shopping Boulevard - BSB  BSB      BR   
4  62ed2de97877dc4c1281d5ed    True   Asa Norte Eixinho - ASN  ASN      BR   

            timezone                                            address  \
0  America/Sao_Paulo                      QD 06 - Área Reservada 3, s/n   
1  America/Sao_Paulo    SPMN EPIA DF 03 Lote 02 Loja 01  Brasília -  DF   
2  America/Sao_Paulo  SHIN CA 4 Lote A, lojas 177/178 – Shopping Igu...   
3  America/Sao_Paulo  STN Terminal Norte, S/N, Conjunto J, Loja T51 ...   
4  America/Sao_Paulo  EQN 208/209 Bloco A – Lj. 50 – Asa Norte  Bras...   

       city neighborhood       cep  ... services.mcDelivery  \
0  Brasília   Sobradinho  73070035  ...               False   
1  Brasília   Lago Norte  71560100  ...                True   
2  Brasília   Lago Norte  71503504  ...               False   
3  Brasília    Asa Norte  70770100  ...                True   
4  Brasília    Asa Norte  70853400  ...                True   

  services.timeExtended services.mcParty services.playPlace  services.parking  \
0                  True            False              False              True   
1                  True            False              False             False   
2                 False            False              False             False   
3                 False            False              False             False   
4                  True            False              False             False   

   services.wifi  services.wheelchairAccess  services.dessertCenter  \
0           True                       True                   False   
1          False                       True                   False   
2          False                       True                    True   
3          False                       True                   False   
4          False                       True                   False   

  services.shoppingCenter                             generalHour.daysOfWeek  
0                   False  [{'day': 'mon', 'timePeriods': [{'start': '08:...  
1                   False  [{'day': 'mon', 'timePeriods': [{'start': '10:...  
2                    True  [{'day': 'mon', 'timePeriods': [{'start': '11:...  
3                    True  [{'day': 'mon', 'timePeriods': [{'start': '10:...  
4                   False  [{'day': 'mon', 'timePeriods': [{'start': '10:...  

[5 rows x 39 columns]

In [190]:
df.describe()

distance  coordinates.longitude  coordinates.latitude  \
count   2343.000000            2343.000000           2343.000000   
mean    1380.530839             -42.518490            -23.901153   
std     1147.258006               9.610560              9.958499   
min       34.005271             -63.876013            -56.114344   
25%      898.639183             -47.068181            -23.694802   
50%      920.855204             -46.546769            -23.480102   
75%     1346.766388             -42.568615            -21.225657   
max    11125.013013              46.701319             23.704880   

       coordinates.lng  coordinates.lat  
count      2343.000000      2343.000000  
mean        -42.518490       -23.901153  
std           9.610560         9.958499  
min         -63.876013       -56.114344  
25%         -47.068181       -23.694802  
50%         -46.546769       -23.480102  
75%         -42.568615       -21.225657  
max          46.701319        23.704880

In [191]:
df.shape

(2343, 39)

In [192]:
rename_dict = {
    'id': 'Identificador',
    'active': 'Ativo',
    'name': 'Nome_do_lugar',
    'code': 'Codigo',
    'country': 'Pais',
    'timezone': 'Fuso_horario',
    'address': 'Endereco',
    'city': 'Cidade',
    'neighborhood': 'Bairro',
    'services.mcDelivery': 'Delivery',
    'services.timeExtended': 'Servicos_com_tempo_extimado',
    'services.mcParty': 'Mcfestas_de_servicos',
    'services.playPlace': 'Lugar_de_brincar',
    'services.parking': 'Serviços_de_estacionamento',
    'services.wifi': 'servicos_com_Wifi',
    'services.wheelchairAccess': 'Acessibilidade',
    'services.dessertCenter': 'Centro_de_sobremesas',
    'services.shoppingCenter': 'Shopping_center',
    'generalHour.daysOfWeek': 'horas_dias_da_semana',
}
df = df.rename(columns=rename_dict)


In [193]:
df.head()

_id  Ativo             Nome_do_lugar Codigo Pais  \
0  61d7fcbda283bebac83e89c9   True          SOBRADINHO - SBR    SBR   BR   
1  62ed34bb7877dc2d6381ea1c   True   Drive Saída Norte - DSN    DSN   BR   
2  62ed31417877dcd78481e3a7   True   Shopping Iguatemi - BSI    BSI   BR   
3  62ed30849b780128a34229f8   True  Shopping Boulevard - BSB    BSB   BR   
4  62ed2de97877dc4c1281d5ed   True   Asa Norte Eixinho - ASN    ASN   BR   

        Fuso_horario                                           Endereco  \
0  America/Sao_Paulo                      QD 06 - Área Reservada 3, s/n   
1  America/Sao_Paulo    SPMN EPIA DF 03 Lote 02 Loja 01  Brasília -  DF   
2  America/Sao_Paulo  SHIN CA 4 Lote A, lojas 177/178 – Shopping Igu...   
3  America/Sao_Paulo  STN Terminal Norte, S/N, Conjunto J, Loja T51 ...   
4  America/Sao_Paulo  EQN 208/209 Bloco A – Lj. 50 – Asa Norte  Bras...   

     Cidade      Bairro       cep  ... Delivery Servicos_com_tempo_extimado  \
0  Brasília  Sobradinho  73070035  ...    False                        True   
1  Brasília  Lago Norte  71560100  ...     True                        True   
2  Brasília  Lago Norte  71503504  ...    False                       False   
3  Brasília   Asa Norte  70770100  ...     True                       False   
4  Brasília   Asa Norte  70853400  ...     True                        True   

  Mcfestas_de_servicos Lugar_de_brincar  Serviços_de_estacionamento  \
0                False            False                        True   
1                False            False                       False   
2                False            False                       False   
3                False            False                       False   
4                False            False                       False   

   servicos_com_Wifi  Acessibilidade  Centro_de_sobremesas Shopping_center  \
0               True            True                 False           False   
1              False            True                 False           False   
2              False            True                  True            True   
3              False            True                 False            True   
4              False            True                 False           False   

                                horas_dias_da_semana  
0  [{'day': 'mon', 'timePeriods': [{'start': '08:...  
1  [{'day': 'mon', 'timePeriods': [{'start': '10:...  
2  [{'day': 'mon', 'timePeriods': [{'start': '11:...  
3  [{'day': 'mon', 'timePeriods': [{'start': '10:...  
4  [{'day': 'mon', 'timePeriods': [{'start': '10:...  

[5 rows x 39 columns]

In [194]:
df['servicos_com_Wifi'] = df['servicos_com_Wifi'].fillna(0)
df['Acessibilidade'] = df['Acessibilidade'].fillna(0)
df = df.dropna(subset=['Cidade'])

print("\nValores nulos após o tratamento:")
print(df[['Cidade', 'Bairro', 'Acessibilidade', 'servicos_com_Wifi', 'Servicos_com_tempo_extimado', 'Delivery']].isnull().sum())




Valores nulos após o tratamento:
Cidade                           0
Bairro                         850
Acessibilidade                   0
servicos_com_Wifi                0
Servicos_com_tempo_extimado    555
Delivery                         0
dtype: int64


In [195]:
print("Valores nulos antes do tratamento:")
print(df[['Cidade', 'Bairro', 'services.breakfast', 'servicos_com_Wifi', 'Acessibilidade']].isnull().sum())
df['services.breakfast'] = df['services.breakfast'].fillna(0)
df['servicos_com_Wifi'] = df['servicos_com_Wifi'].fillna(0)
df['Acessibilidade'] = df['Acessibilidade'].fillna(0)
df = df.dropna(subset=['Bairro'])
print("\nValores nulos após o tratamento:")
print(df[['Cidade', 'Bairro', 'services.breakfast', 'servicos_com_Wifi', 'Acessibilidade']].isnull().sum())




Valores nulos antes do tratamento:
Cidade                  0
Bairro                850
services.breakfast    555
servicos_com_Wifi       0
Acessibilidade          0
dtype: int64

Valores nulos após o tratamento:
Cidade                0
Bairro                0
services.breakfast    0
servicos_com_Wifi     0
Acessibilidade        0
dtype: int64


In [196]:
features = ['Cidade', 'Bairro', 'servicos_com_wifi', 'Acessibilidade', 'Delivery', 'Servicos_com_tempo_extimado']
target = 'Delivery' 


In [197]:
le = LabelEncoder()
df['Cidade'] = le.fit_transform(df['Cidade'])
df['Bairro'] = le.fit_transform(df['Bairro'])


In [198]:
import pickle
from sklearn.preprocessing import LabelEncoder

# Crie e ajuste encoders separados para Cidade e Bairro
le_cidade = LabelEncoder()
le_bairro = LabelEncoder()
le_cidade.fit(df['Cidade'])
le_bairro.fit(df['Bairro'])

with open('le_cidade.pkl', 'wb') as file:
    pickle.dump(le_cidade, file)
with open('le_bairro.pkl', 'wb') as file:
    pickle.dump(le_bairro, file)



In [199]:
from sklearn.impute import SimpleImputer

# Corrigir o nome da coluna para corresponder ao DataFrame
features_corrigido = ['Cidade', 'Bairro', 'servicos_com_Wifi', 'Acessibilidade', 'Delivery', 'Servicos_com_tempo_extimado']
X = df[features_corrigido]
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
y = df[target].map({True: 1, False: 0})



In [200]:
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)


In [201]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)



In [202]:
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)




In [203]:
param_grid = {
    'n_estimators': [200, 250, 300],
    'max_depth': [10, 12, 15],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 5],
    'class_weight': ['balanced', None]
}

# Realize o GridSearch
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_resampled_scaled, y_resampled)



GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'class_weight': ['balanced', None],
                         'max_depth': [10, 12, 15], 'min_samples_leaf': [2, 5],
                         'min_samples_split': [5, 10],
                         'n_estimators': [200, 250, 300]},
             scoring='f1_macro')

In [204]:
rf = grid_search.best_estimator_

# Faça as previsões
y_pred = rf.predict(X_test_scaled)



In [205]:
print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred))
print("Relatório de classificação:")
print(classification_report(y_test, y_pred))
print("Acurácia:", accuracy_score(y_test, y_pred))



Matriz de confusão:
[[218   0]
 [  0  81]]
Relatório de classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       218
           1       1.00      1.00      1.00        81

    accuracy                           1.00       299
   macro avg       1.00      1.00      1.00       299
weighted avg       1.00      1.00      1.00       299

Acurácia: 1.0


In [206]:
with open('treinamentomc.pkl', 'wb') as file:
    pickle.dump(rf, file)
